In [ ]:
# !pip install grad-cam
!pip install ttach
!pip install timm

     |████████████████████████████████| 376 kB 5.0 MB/s 


In [ ]:
!git clone https://github.com/vigneshuw/pytorch-grad-cam.git
!pip install pytorch-grad-cam/

Cloning into 'pytorch-grad-cam'...
remote: Enumerating objects: 702, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 702 (delta 155), reused 129 (delta 129), pack-reused 501
Receiving objects: 100% (702/702), 1.89 MiB | 21.02 MiB/s, done.
Resolving deltas: 100% (388/388), done.
Processing ./pytorch-grad-cam
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 60.3 MB 1.1 MB/s 
  Created wheel for grad-cam: filename=gra

In [ ]:
# Google Collab Requirements / Checks
from google.colab.patches import cv2_imshow # Change this before running elsewhere!!

from google.colab import drive
drive.mount('/content/gdrive')

# Imports
import os
import cv2
import glob
import gc
import numpy as np
import pandas as pd
import copy

from PIL import Image

import torch
from torchvision import transforms
from torchvision.models import resnet50
from torchsummary import summary

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

Mounted at /content/gdrive


In [ ]:
# Load manual ms coco dataset and bounding boxes
# Varun's Directory
# dir_uri = '/content/gdrive/MyDrive/University of Wisconsin-Madison/2021 Fall/cs762'
# Devesh's Directory
dir_uri = '/content/gdrive/MyDrive'

proj_dir = dir_uri + '/CS762_Deep_Learning_Project'
df = pd.read_excel( proj_dir + '/fullBBOX.xlsx')

dataset_folder = dir_uri + '/CS762_Deep_Learning_Project/datasets/Manual_MSCOCO' 
image_names = []
for filename in os.listdir(dataset_folder):
  image_names.append(filename)

image_names.sort(key=lambda x: int(x.split('-')[0]))

# Load iamge net categories
imageNet_labels_path = dir_uri + "/CS762_Deep_Learning_Project/datasets/imagenet_classes.txt"
# Read the categories
with open(imageNet_labels_path, "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Load bbox's
bbox_path = proj_dir + '/fullBBOX.xlsx'
bbox_df = pd.read_excel(bbox_path)
all_boxes = bbox_df.to_numpy()

output_file = proj_dir + '/Q2/q2_output_resnet_deit.csv'
headers = ['image_name', 'threshold', 'resnet label', 'resnet accuracy', 'resnet whole image %', 'resnet in bbox %', 'resnet outside bbox %', 'deit label', 'deit accuracy', 'deit whole image %', 'deit in bbox %', 'deit outside bbox %']

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize( (224, 224) ),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def reshape_transform(tensor, height=14, width=14):
    result = tensor[:, 1:, :].reshape(tensor.size(0),
                                      height, width, tensor.size(2))

    # Bring the channels to the first dimension,
    # like in CNNs.
    result = result.transpose(2, 3).transpose(1, 2)
    return result


# bbox given as one row from our fullBBox.xlsx file
def eval_metric_percent_above(grayscale_cam, bbox, threshold):

    # find area of bbox
    topLeft_w = int(bbox[2])
    topLeft_h = int(bbox[3])
    botRight_w = int(bbox[6])
    botRight_h = int(bbox[7])
    width_bbox = abs(botRight_w - topLeft_w)
    height_bbox = abs(topLeft_h - botRight_h)
    area = width_bbox * height_bbox

    # percent above threshold whole image
    above_threshold = (grayscale_cam > threshold)
    count_above_threshold = above_threshold.sum()
    total_count = grayscale_cam.shape[0] * grayscale_cam.shape[1]
    percent_above_threshold_whole_image = (count_above_threshold / total_count) * 100

    # percent above threshold in bbox
    bbox_grayscale_cam = grayscale_cam[topLeft_h:botRight_h, topLeft_w:botRight_w]
    bbox_above_threshold = (bbox_grayscale_cam > threshold)
    bbox_count_above_threshold = bbox_above_threshold.sum()
    bbox_total_count = bbox_grayscale_cam.shape[0] * bbox_grayscale_cam.shape[1]
    percent_above_threshold_in_bbox = (bbox_count_above_threshold / bbox_total_count) * 100

    # percent above threshold outside bbox
    outside_bbox_cam = copy.deepcopy(grayscale_cam)
    outside_bbox_cam[topLeft_h:botRight_h, topLeft_w:botRight_w] = np.zeros((height_bbox, width_bbox))
    outside_bbox_above_threshold = (outside_bbox_cam > threshold)
    outside_bbox_count_above_threshold = outside_bbox_above_threshold.sum()
    outside_bbox_total_count = total_count - bbox_total_count
    percent_above_threshold_outside_bbox = (outside_bbox_count_above_threshold / outside_bbox_total_count) * 100

    # return
    return percent_above_threshold_whole_image, percent_above_threshold_in_bbox, percent_above_threshold_outside_bbox


def resizeBbox(orig_bbox, orig_img_size, new_img_size):
    orig_h = orig_img_size[0]
    orig_w = orig_img_size[1]
    new_h = new_img_size[0]
    new_w = new_img_size[1]

    h_ratio = new_h / orig_h
    w_ratio = new_w / orig_w

    new_bbox = copy.deepcopy(orig_bbox)
    new_bbox[2] = orig_bbox[2] * w_ratio
    new_bbox[3] = orig_bbox[3] * h_ratio
    new_bbox[4] = orig_bbox[4] * w_ratio
    new_bbox[5] = orig_bbox[5] * h_ratio
    new_bbox[6] = orig_bbox[6] * w_ratio
    new_bbox[7] = orig_bbox[7] * h_ratio
    new_bbox[8] = orig_bbox[8] * w_ratio
    new_bbox[9] = orig_bbox[9] * h_ratio

    return new_bbox

In [ ]:
preprocess_eval = transforms.Compose([
    transforms.Resize( (224, 224) ),
])

def show_cam_on_image_updated(img: np.ndarray,
                      mask: np.ndarray,
                      use_rgb: bool = False,
                      colormap: int = cv2.COLORMAP_JET) -> np.ndarray:
    """ This function overlays the cam mask on the image as an heatmap.
    By default the heatmap is in BGR format.
    :param img: The base image in RGB or BGR format.
    :param mask: The cam mask.
    :param use_rgb: Whether to use an RGB or BGR heatmap, this should be set to True if 'img' is in RGB format.
    :param colormap: The OpenCV colormap to be used.
    :returns: The default image with the cam overlay.
    """
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), colormap)
    if use_rgb:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = np.float32(heatmap) / 255

    if np.max(img) > 1:
        raise Exception(
            "The input image should np.float32 in the range [0, 1]")

    cam = heatmap + img
    cam = cam / np.max(cam)
    return heatmap, np.uint8(255 * cam)


def visualize_eval_metric(rgb_image, grayscale_cam, heatmap, bbox, threshold):
    red_color = (0, 0, 255)
    thickness = 2
    start_point = (int(bbox[2]), int(bbox[3]))
    end_point = (int(bbox[6]), int(bbox[7]))

    above_threshold = (grayscale_cam > threshold)

    for i in range(grayscale_cam.shape[0]):
      for j in range(grayscale_cam.shape[1]):
        if(above_threshold[i, j] == False):
          heatmap[i,j,0] = 0
          heatmap[i,j,1] = 0
          heatmap[i,j,2] = 0

    cam = heatmap + rgb_image
    cam = cam / np.max(cam)
    cam = np.uint8(255 * cam)
    cam = cv2.rectangle(cam, start_point, end_point, red_color, thickness)
    return cam

In [ ]:
def show_cam_on_image_updated_blank(img: np.ndarray,
                      mask: np.ndarray,
                      use_rgb: bool = False,
                      colormap: int = cv2.COLORMAP_JET) -> np.ndarray:
    """ This function overlays the cam mask on the image as an heatmap.
    By default the heatmap is in BGR format.
    :param img: The base image in RGB or BGR format.
    :param mask: The cam mask.
    :param use_rgb: Whether to use an RGB or BGR heatmap, this should be set to True if 'img' is in RGB format.
    :param colormap: The OpenCV colormap to be used.
    :returns: The default image with the cam overlay.
    """
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), colormap)
    if use_rgb:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = np.float32(heatmap) / 255

    if np.max(img) > 1:
        raise Exception(
            "The input image should np.float32 in the range [0, 1]")

    img[(img[:,:,0]<300) & (img[:,:,1]<300) & (img[:,:,2]<300)] = [1,1,1]
    cam = heatmap + img
    cam = cam / np.max(cam)
    return heatmap, np.uint8(255 * cam)

In [ ]:
resnet_model = resnet50(pretrained=True)
resnet_model.eval();

deit_model = torch.hub.load('facebookresearch/deit:main',
                          'deit_tiny_patch16_224', pretrained=True)
deit_model.eval();

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Downloading: "https://github.com/facebookresearch/deit/archive/main.zip" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth" to /root/.cache/torch/hub/checkpoints/deit_tiny_patch16_224-a1311bcf.pth


  0%|          | 0.00/21.9M [00:00<?, ?B/s]

In [ ]:
resnet_target_layers = [resnet_model.layer4[-1]]
deit_target_layers = [deit_model.blocks[-1].norm1]

target_category = None

resnet_input_image_shape = (224, 224, 3)
deit_input_image_shape = (224, 224, 3)

threshold_options = [0.7, 0.9]

In [ ]:
output_data = []

for threshold in threshold_options:
  for i in range( len( image_names ) ):
    # Create input
    test_img = Image.open( dataset_folder + '/' + image_names[i] ).convert('RGB')
    input_tensor = preprocess(test_img)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    orig_bbox = all_boxes[i]
    orig_img_shape = np.array(test_img).shape

    # Create target lable
    target_name = image_names[i].split( '-' )[1]
    target_name = target_name.split( '.' )[0]

    # Set target category
    target_category = [index for index, class_instance in enumerate(categories) if class_instance == target_name]

    # If target category is empty (ie label does not exist in image net, skip this image):
    if ( len( target_category ) == 0 ):
      continue

    print( image_names[i] )

    # RESNET

    with torch.no_grad():
      output = resnet_model(input_batch)

    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Get top category
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    resnet_predicted_label = categories[top5_catid[0]]
    resnet_predicted_label_prob = top5_prob[0].item()

    resnet_cam = GradCAM(model=resnet_model, target_layers=resnet_target_layers)
    resnet_grayscale_cam = resnet_cam(input_tensor=input_batch, target_category=target_category)
    resnet_grayscale_cam = resnet_grayscale_cam[0, :]
    
    # resize bbox and calculate metrics
    new_bbox_resnet = resizeBbox(orig_bbox, (orig_img_shape[0], orig_img_shape[1]), (resnet_input_image_shape[0], resnet_input_image_shape[1]))
    resnet_whole_img, resnet_in_bbox, resnet_out_bbox = eval_metric_percent_above(resnet_grayscale_cam, new_bbox_resnet, threshold)


    # DEIT

    # compute the predictions
    with torch.no_grad():
      output = deit_model(input_batch)

    # and convert them into probabilities
    scores = torch.nn.functional.softmax(output, dim=-1)[0]

    # finally get the index of the prediction with highest score
    topk_scores, topk_label = torch.topk(scores, k=5, dim=-1)
    deit_predicted_label = categories[topk_label[0]]
    deit_predicted_label_prob = topk_scores[0].item()

    deit_cam = GradCAM(model=deit_model, target_layers=deit_target_layers, reshape_transform=reshape_transform)
    deit_grayscale_cam = deit_cam(input_tensor=input_batch, target_category=target_category)
    deit_grayscale_cam = deit_grayscale_cam[0, :]

    # resize bbox and calculate metrics
    new_bbox_deit = resizeBbox(orig_bbox, (orig_img_shape[0], orig_img_shape[1]), (deit_input_image_shape[0], deit_input_image_shape[1]))
    deit_whole_img, deit_in_bbox, deit_out_bbox = eval_metric_percent_above(deit_grayscale_cam, new_bbox_deit, threshold)

    output_data.append( [image_names[i], threshold, resnet_predicted_label, resnet_predicted_label_prob, resnet_whole_img, resnet_in_bbox, resnet_out_bbox, deit_predicted_label, deit_predicted_label_prob, deit_whole_img, deit_in_bbox, deit_out_bbox] )

    # Delete garbage for ram
    gc.collect()

50-zebra.jpg
<class 'torch.Tensor'>
The max logit value is [tensor(15.8709, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(9.0648, grad_fn=<AddBackward0>)]
51-zebra.jpg
<class 'torch.Tensor'>
The max logit value is [tensor(13.7910, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(8.9930, grad_fn=<AddBackward0>)]
52-zebra.jpg
<class 'torch.Tensor'>
The max logit value is [tensor(20.2493, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(8.7388, grad_fn=<AddBackward0>)]
53-zebra.jpg
<class 'torch.Tensor'>
The max logit value is [tensor(19.4557, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(9.0184, grad_fn=<AddBackward0>)]
54-zebra.jpg
<class 'torch.Tensor'>
The max logit value is [tensor(17.0945, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(8.9074, grad_fn=<AddBackward0>)]
55-backpack.jpg
<class 'torch.Tensor'>
The max logit value i

In [ ]:
df = pd.DataFrame (output_data, columns=headers)
df.to_csv(output_file)

In [ ]:
output_images_dir = proj_dir + '/Q2/output_images/'
images_to_visualize = [50, 56, 60, 63, 67]
threshold = 0.7
generate_vis_analysis = True

In [ ]:
if(generate_vis_analysis):
  for i in range(0, len(image_names)):
    # Create input
    test_img = Image.open( dataset_folder + '/' + image_names[i] ).convert('RGB')
    resnet_path = output_images_dir + image_names[i] + '_resnet_whole'
    resnet_path_t = output_images_dir + image_names[i] + '_resnet_thres'
    deit_path = output_images_dir + image_names[i] + '_deit_whole'
    deit_path_t = output_images_dir + image_names[i] + '_deit_thres'
    input_tensor = preprocess(test_img)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    orig_bbox = all_boxes[i]
    orig_img_shape = np.array(test_img).shape

    rgb_image = np.array(preprocess_eval(test_img)).astype(np.float32)
    rgb_image = rgb_image/255

    # Create target lable
    target_name = image_names[i].split( '-' )[1]
    target_name = target_name.split( '.' )[0]

    # Set target category
    target_category = [index for index, class_instance in enumerate(categories) if class_instance == target_name]

    # If target category is empty (ie label does not exist in image net, skip this image):
    if ( len( target_category ) == 0 ):
      continue

    # Resnet
    resnet_cam = GradCAM(model=resnet_model, target_layers=resnet_target_layers)
    resnet_grayscale_cam = resnet_cam(input_tensor=input_batch, target_category=target_category)
    resnet_grayscale_cam = resnet_grayscale_cam[0, :]

    resnet_heatmap, resnet_visualization = show_cam_on_image_updated(rgb_image, resnet_grayscale_cam, use_rgb=False)
    cv2.imwrite(resnet_path, resnet_visualization)

    new_bbox_resnet = resizeBbox(orig_bbox, (orig_img_shape[0], orig_img_shape[1]), (resnet_input_image_shape[0], resnet_input_image_shape[1]))
    resnet_visualized_cam_threshold = visualize_eval_metric(rgb_image, resnet_grayscale_cam, resnet_heatmap, new_bbox_resnet, threshold)
    cv2.imwrite(resnet_path_t, resnet_visualized_cam_threshold)

    # DEIT
    deit_cam = GradCAM(model=deit_model, target_layers=deit_target_layers, reshape_transform=reshape_transform)
    deit_grayscale_cam = deit_cam(input_tensor=input_batch, target_category=target_category)
    deit_grayscale_cam = deit_grayscale_cam[0, :]

    deit_heatmap, deit_visualization = show_cam_on_image_updated(rgb_image, deit_grayscale_cam, use_rgb=False)
    cv2.imwrite(deit_path, deit_visualization)

    new_bbox_deit = resizeBbox(orig_bbox, (orig_img_shape[0], orig_img_shape[1]), (deit_input_image_shape[0], deit_input_image_shape[1]))
    deit_visualized_cam_threshold = visualize_eval_metric(rgb_image, deit_grayscale_cam, deit_heatmap, new_bbox_deit, threshold)
    cv2.imwrite(deit_path_t, deit_visualized_cam_threshold)



<class 'torch.Tensor'>
The max logit value is [tensor(15.8709, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(9.0648, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(13.7910, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(8.9930, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(20.2493, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(8.7388, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(19.4557, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(9.0184, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(17.0945, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(8.9074, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(13.4308, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit 

In [ ]:
output_images_dir = proj_dir + '/Q2/output_images_blank_background/'
generate_vis_analysis = True

if(generate_vis_analysis):
  for i in range(0, len(image_names)):
    # Create input
    test_img = Image.open( dataset_folder + '/' + image_names[i] ).convert('RGB')
    resnet_path = output_images_dir + 'resnet/' + image_names[i].split('.')[0] + '_resnet_whole.' + image_names[i].split('.')[1]
    deit_path = output_images_dir + 'deit/' + image_names[i].split('.')[0] + '_deit_whole.' + image_names[i].split('.')[1]

    input_tensor = preprocess(test_img)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    orig_bbox = all_boxes[i]
    orig_img_shape = np.array(test_img).shape

    rgb_image = np.array(preprocess_eval(test_img)).astype(np.float32)
    rgb_image = rgb_image/255

    # Create target lable
    target_name = image_names[i].split( '-' )[1]
    target_name = target_name.split( '.' )[0]

    # Set target category
    target_category = [index for index, class_instance in enumerate(categories) if class_instance == target_name]

    # If target category is empty (ie label does not exist in image net, skip this image):
    if ( len( target_category ) == 0 ):
      continue

    # Resnet
    resnet_cam = GradCAM(model=resnet_model, target_layers=resnet_target_layers)
    resnet_grayscale_cam = resnet_cam(input_tensor=input_batch, target_category=target_category)
    resnet_grayscale_cam = resnet_grayscale_cam[0, :]

    resnet_heatmap, resnet_visualization = show_cam_on_image_updated_blank(rgb_image, resnet_grayscale_cam, use_rgb=False)
    cv2.imwrite(resnet_path, resnet_visualization)

    # DEIT
    deit_cam = GradCAM(model=deit_model, target_layers=deit_target_layers, reshape_transform=reshape_transform)
    deit_grayscale_cam = deit_cam(input_tensor=input_batch, target_category=target_category)
    deit_grayscale_cam = deit_grayscale_cam[0, :]

    deit_heatmap, deit_visualization = show_cam_on_image_updated_blank(rgb_image, deit_grayscale_cam, use_rgb=False)
    cv2.imwrite(deit_path, deit_visualization)

<class 'torch.Tensor'>
The max logit value is [tensor(15.8709, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(9.0648, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(13.7910, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(8.9930, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(20.2493, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(8.7388, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(19.4557, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(9.0184, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(17.0945, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(8.9074, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit value is [tensor(13.4308, grad_fn=<AddBackward0>)]
<class 'torch.Tensor'>
The max logit 